In [124]:
gdbs = list()
for dirpath, dirs, files in os.walk(r'scenario'):
    for d in dirs:
        dir_name = os.path.join(base_path, dirpath, d)
        if os.path.isdir(dir_name) and dir_name.endswith('.gdb'):
            if not d == 'scenario_template.gdb':
                gdbs.append(os.path.join(dirpath, d))

In [ ]:
for mode in ['auto', 'bike', 'transit']:
    scenario_folder = os.path.join('scenario', mode)
    for file in os.listdir(scenario_folder):
        d = os.path.join(base_path, scenario_folder, file)
        if os.path.isdir(d) and dir_name.endswith('.gdb'):
            if arcpy.Exists(os.path.join(d, 'scores')) == False:
                scoring.score(gdb, mode = mode)

C:\wfrc\ato\scenario\auto\.gitignore
C:\wfrc\ato\scenario\auto\new_2700_west_overpass.gdb
C:\wfrc\ato\scenario\auto\new_5600_west.gdb
C:\wfrc\ato\scenario\auto\new_construction_1250_west.gdb
C:\wfrc\ato\scenario\auto\new_construct_1200_west.gdb
C:\wfrc\ato\scenario\auto\new_highland.gdb
C:\wfrc\ato\scenario\auto\widen_5600_south.gdb
C:\wfrc\ato\scenario\auto\widen_9000_south.gdb
C:\wfrc\ato\scenario\bike\.gitignore
C:\wfrc\ato\scenario\transit\.gitignore
C:\wfrc\ato\scenario\transit\davis_brt.gdb
Solving skim using network scenario\transit\davis_brt.gdb\NetworkDataset\NetworkDataset_ND


In [115]:
import importlib
importlib.reload(scoring)

<module 'ato_tools.scoring' from 'C:\\wfrc\\ato\\src\\ato_tools\\scoring.py'>

In [111]:
if 'ato_tools' in sys.modules:
    import importlib
    importlib.reload(scoring)